# DWH static tables

## dim_currency

In [ ]:
CREATE TABLE IF NOT EXISTS dim_currency (
    currency_id SERIAL PRIMARY KEY,
    currency_code VARCHAR(3) NOT NULL
);

INSERT INTO dim_currency (currency_id, currency_code) VALUES
(1, 'CHF'),
(2, 'EUR');

## dim_time

In [ ]:
CREATE TABLE IF NOT EXISTS dim_time (
    time_id SERIAL PRIMARY KEY,
    timestamp_utc TIMESTAMP WITHOUT TIME ZONE NOT NULL,
    date DATE NOT NULL,
    hour INTEGER CHECK (hour >= 0 AND hour <= 23),
    day_of_week VARCHAR(3) CHECK (day_of_week IN (
        'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'
    )),
    month VARCHAR(3) CHECK (month IN (
        'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
    )),
    year INTEGER CHECK (year >= 1900)
);

WITH RECURSIVE time_gen AS (
    SELECT 
        TIMESTAMP '2024-01-01 00:00:00' AS ts
    UNION ALL
    SELECT 
        ts + INTERVAL '1 hour'
    FROM time_gen
    WHERE ts + INTERVAL '1 hour' <= TIMESTAMP '2025-06-30 23:00:00'
)
INSERT INTO dim_time (
    timestamp_utc,
    date,
    hour,
    day_of_week,
    month,
    year
)
SELECT
    ts AS timestamp_utc,
    ts::date AS date,
    EXTRACT(HOUR FROM ts)::int AS hour,
    TO_CHAR(ts, 'Dy') AS day_of_week,
    TO_CHAR(ts, 'Mon') AS month,
    EXTRACT(YEAR FROM ts)::int AS year
FROM time_gen;
